## Overview

The EMISOpen consultation writeback is essentially the same as HL7 v2 ORU Unsolicited Observation.

| HL7 v2.5.1 Segment                                                      | HL7 FHIR Resource                                                                               | EMISOpen                             |
|-------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------|--------------------------------------|
| [PID](https://hl7-definition.caristix.com/v2/HL7v2.5.1/Segments/PID)    | [Patient.identifier](https://virtually-healthcare.github.io/R4/StructureDefinition-Patient.html)             | RegistrationType (extends IdentType) |
| [ORC](https://hl7-definition.caristix.com/v2/HL7v2.5.1/Segments/ORC)    | [DiagnosticReport](hhttps://virtually-healthcare.github.io/R4/StructureDefinition-DiagnosticReport.html)      | ConsultationListType                 |
| - [OBR](https://hl7-definition.caristix.com/v2/HL7v2.5.1/Segments/OBR)  | [Observation-Panel](https://virtually-healthcare.github.io/R4/StructureDefinition-Observation-Panel.html)    |                                      |
| -- [NTE](https://hl7-definition.caristix.com/v2/HL7v2.5.1/Segments/NTE) | [DiagnosticReport.note](https://virtually-healthcare.github.io/R4/StructureDefinition-DiagnosticReport.html) |                                      |
| -- [OBX](https://hl7-definition.caristix.com/v2/HL7v2.5.1/Segments/OBX) | [Observation](https://virtually-healthcare.github.io/R4/StructureDefinition-Observation.html)           |                                      |